<a href="https://colab.research.google.com/github/nhwhite212/DealingwithDataSpring2021/blob/colab/1-SQL/G1-Inserting_Data_in_MySQL_using_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Inserting data in MySQL using Python

First let's start with a basic piece of code that fetches the data that we want to insert in the database. For our example, we will get the data about the Citibike stations, using the correspoding API call provided by the Citibike website:

In [1]:
import requests

In [2]:
# Let's get the most recent data from the Citibike API
url = 'http://www.citibikenyc.com/stations/json'
results = requests.get(url).json() 
print(results)

{'executionTime': '2016-01-22 04:32:49 PM', 'stationBeanList': [{'id': 72, 'stationName': 'W 52 St & 11 Ave', 'availableDocks': 32, 'totalDocks': 39, 'latitude': 40.76727216, 'longitude': -73.99392888, 'statusValue': 'In Service', 'statusKey': 1, 'availableBikes': 7, 'stAddress1': 'W 52 St & 11 Ave', 'stAddress2': '', 'city': '', 'postalCode': '', 'location': '', 'altitude': '', 'testStation': False, 'lastCommunicationTime': '2016-01-22 04:30:15 PM', 'landMark': ''}, {'id': 79, 'stationName': 'Franklin St & W Broadway', 'availableDocks': 0, 'totalDocks': 33, 'latitude': 40.71911552, 'longitude': -74.00666661, 'statusValue': 'In Service', 'statusKey': 1, 'availableBikes': 33, 'stAddress1': 'Franklin St & W Broadway', 'stAddress2': '', 'city': '', 'postalCode': '', 'location': '', 'altitude': '', 'testStation': False, 'lastCommunicationTime': '2016-01-22 04:32:41 PM', 'landMark': ''}, {'id': 82, 'stationName': 'St James Pl & Pearl St', 'availableDocks': 27, 'totalDocks': 27, 'latitude': 

In [3]:
# We only need a subset of the data in the JSON returned by the Citibike API, so we keep only we need
print(results.keys())
print()
data = results["stationBeanList"]
print(data[0:3])

dict_keys(['executionTime', 'stationBeanList'])

[{'id': 72, 'stationName': 'W 52 St & 11 Ave', 'availableDocks': 32, 'totalDocks': 39, 'latitude': 40.76727216, 'longitude': -73.99392888, 'statusValue': 'In Service', 'statusKey': 1, 'availableBikes': 7, 'stAddress1': 'W 52 St & 11 Ave', 'stAddress2': '', 'city': '', 'postalCode': '', 'location': '', 'altitude': '', 'testStation': False, 'lastCommunicationTime': '2016-01-22 04:30:15 PM', 'landMark': ''}, {'id': 79, 'stationName': 'Franklin St & W Broadway', 'availableDocks': 0, 'totalDocks': 33, 'latitude': 40.71911552, 'longitude': -74.00666661, 'statusValue': 'In Service', 'statusKey': 1, 'availableBikes': 33, 'stAddress1': 'Franklin St & W Broadway', 'stAddress2': '', 'city': '', 'postalCode': '', 'location': '', 'altitude': '', 'testStation': False, 'lastCommunicationTime': '2016-01-22 04:32:41 PM', 'landMark': ''}, {'id': 82, 'stationName': 'St James Pl & Pearl St', 'availableDocks': 27, 'totalDocks': 27, 'latitude': 40.71117416, '

In [4]:
len(data)

509

Now we will connect to our MySQL server. We will use the MySQLdb library of Python.

If you do not have the library, you need to install it by typing in the shell:

`sudo apt-get install python-mysqldb`


In [8]:
!sudo apt-get install -y python-dev libmysqlclient-dev && sudo pip install mysqlclient

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
The following NEW packages will be installed:
  libmysqlclient-dev
0 upgraded, 1 newly installed, 0 to remove and 13 not upgraded.
Need to get 988 kB of archives.
After this operation, 6,013 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmysqlclient-dev amd64 5.7.32-0ubuntu0.18.04.1 [988 kB]
Fetched 988 kB in 2s (540 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting 

In [10]:
import MySQLdb as mdb

#con = mdb.connect(host = 'localhost', 
#                  user = 'root', 
#                  passwd = 'dwdstudent2015', 
#                  charset='utf8', use_unicode=True);


con = mdb.connect(host = 'bigdata.stern.nyu.edu', 
                  user = 'DealingS21',                  
                  passwd = 'DealingS21!!', 
                  charset='utf8', use_unicode=True);

Once we have connected successfully, we need to create our database:

In [11]:
# Query to create a database
db_name = 'citibike_mysql_test'
# Need to drop database since the not exists option doesn't sem to work
db_drop ="drop database {db} ".format(db=db_name)
cursor=con.cursor()
cursor.execute(db_drop)
cursor.close()

OperationalError: ignored

In [13]:
create_db_query = "CREATE DATABASE IF NOT EXISTS {db} DEFAULT CHARACTER SET 'utf8'".format(db=db_name)

# Create a database
cursor = con.cursor()
cursor.execute(create_db_query)
cursor.close()

Then we create the table where we will store our data. For our example, we will just import three fields in the database: station_id, station_name, and number_of_docks

In [14]:
cursor = con.cursor()
table_name = 'Docks'
# Create a table
# Drop it if it exists
#create_drop_docks=''' drop table {db}.{table}'''.format(db=db_name,table=table_name)
#cursor.execute(create_drop_docks)

# The {db} and {table} are placeholders for the parameters in the format(....) statement
create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table} 
                                (station_id int, 
                                station_name varchar(250), 
                                number_of_docks int,
                                available_docks int,
                                date datetime,
                                PRIMARY KEY(station_id, date)
                                )'''.format(db=db_name, table=table_name)
cursor.execute(create_table_query)
cursor.close()

Finally, we import the data into our table, using the INSERT command. 

In [15]:
from datetime import date, datetime, timedelta


query_template = '''INSERT INTO {db}.{table}(station_id, 
                                            station_name, 
                                            number_of_docks, 
                                            available_docks, 
                                            date) 
                    VALUES (%s, %s, %s, %s, %s)'''.format(db=db_name, table=table_name)
cursor = con.cursor()

# THIS IS PROHIBITED
# query = "INSERT INTO citibike.Docks(station_id, station_name, number_of_docks) VALUES ("+entry["id"]+", "+entry["stationName"]+", "+entry["totalDocks"]+")"
for entry in data:
    dockid = entry["id"]
    addr = entry["stationName"]
    docks = entry["totalDocks"]
    available = entry["availableDocks"]
    # date =  datetime.now()
    # lastcommunicationtime is a string of 
    # the form "2016-02-09 10:16:49 AM"
    # See https://docs.python.org/2/library/datetime.html#strftime-and-strptime-behavior
    # to see the documentation on how to parse 
    date = datetime.strptime(entry["lastCommunicationTime"], 
                             '%Y-%m-%d %I:%M:%S %p')
    print("Inserting station", dockid, "at", addr)
    query_parameters = (dockid, addr, docks, available, date)
    cursor.execute(query_template, query_parameters)
# commit the changes

con.commit()
cursor.close()

Inserting station 72 at W 52 St & 11 Ave
Inserting station 79 at Franklin St & W Broadway
Inserting station 82 at St James Pl & Pearl St
Inserting station 83 at Atlantic Ave & Fort Greene Pl
Inserting station 116 at W 17 St & 8 Ave
Inserting station 119 at Park Ave & St Edwards St
Inserting station 120 at Lexington Ave & Classon Ave
Inserting station 127 at Barrow St & Hudson St
Inserting station 128 at MacDougal St & Prince St
Inserting station 137 at E 56 St & Madison Ave
Inserting station 143 at Clinton St & Joralemon St
Inserting station 144 at Nassau St & Navy St
Inserting station 146 at Hudson St & Reade St
Inserting station 147 at Greenwich St & Warren St
Inserting station 150 at E 2 St & Avenue C
Inserting station 151 at Cleveland Pl & Spring St
Inserting station 152 at Warren St & Church St
Inserting station 153 at E 40 St & 5 Ave
Inserting station 157 at Henry St & Atlantic Ave
Inserting station 160 at E 37 St & Lexington Ave
Inserting station 161 at LaGuardia Pl & W 3 St
Ins

In [16]:
cur = con.cursor(mdb.cursors.DictCursor)
cur.execute("SELECT * FROM {db}.{table}".format(db=db_name, table=table_name))
print(cur.execute)
rows = cur.fetchall()
cur.close()

<bound method BaseCursor.execute of <MySQLdb.cursors.DictCursor object at 0x7f7297003e80>>


In [17]:
for row in rows:
    print("Station ID:", row["station_id"])
    print("Station Name:", row["station_name"])
    print("Number of Docks:", row["number_of_docks"])
    print("Available Docks:", row["available_docks"])
    print("Last Communication:", row["date"])
    print("=============================================")
    


Station ID: 72
Station Name: W 52 St & 11 Ave
Number of Docks: 39
Available Docks: 32
Last Communication: 2016-01-22 16:30:15
Station ID: 79
Station Name: Franklin St & W Broadway
Number of Docks: 33
Available Docks: 0
Last Communication: 2016-01-22 16:32:41
Station ID: 82
Station Name: St James Pl & Pearl St
Number of Docks: 27
Available Docks: 27
Last Communication: 2016-01-22 16:29:41
Station ID: 83
Station Name: Atlantic Ave & Fort Greene Pl
Number of Docks: 62
Available Docks: 21
Last Communication: 2016-01-22 16:32:33
Station ID: 116
Station Name: W 17 St & 8 Ave
Number of Docks: 39
Available Docks: 19
Last Communication: 2016-01-22 16:32:32
Station ID: 119
Station Name: Park Ave & St Edwards St
Number of Docks: 19
Available Docks: 15
Last Communication: 2016-01-22 16:32:29
Station ID: 120
Station Name: Lexington Ave & Classon Ave
Number of Docks: 19
Available Docks: 17
Last Communication: 2016-01-22 16:32:06
Station ID: 127
Station Name: Barrow St & Hudson St
Number of Docks: 31

We can, of course, transform the results back into a DataFrame (see below) or we can use the data directly from the rows object (which is a tuple, containing one dictionary object for each line of the results).

In [18]:
import pandas as pd
cur = con.cursor(mdb.cursors.DictCursor)
cur.execute("SELECT * FROM {db}.{table}".format(db=db_name, table=table_name))
rows = cur.fetchall()
cur.close()

In [19]:
df_from_sql = pd.DataFrame(list(rows))
df_from_sql

,station_id,station_name,number_of_docks,available_docks,date
0,72,W 52 St & 11 Ave,39,32,2016-01-22 16:30:15
1,79,Franklin St & W Broadway,33,0,2016-01-22 16:32:41
2,82,St James Pl & Pearl St,27,27,2016-01-22 16:29:41
3,83,Atlantic Ave & Fort Greene Pl,62,21,2016-01-22 16:32:33
4,116,W 17 St & 8 Ave,39,19,2016-01-22 16:32:32
...,...,...,...,...,...
504,3238,E 80 St & 2 Ave,37,35,2016-01-22 16:30:37
505,3241,Monroe St & Tompkins Ave,19,18,2016-01-22 16:30:38
506,3242,Schermerhorn St & Court St,39,5,2016-01-22 16:31:33
507,3243,E 58 St & 1 Ave,29,28,2016-01-22 16:30:05


In [20]:
# We can then compute functions directly on the dataframe
sum(df_from_sql["number_of_docks"])

16685

In [21]:
# We can then compute functions directly on the dataframe
sum(df_from_sql["available_docks"])

10945

In [22]:
# And we can also create 
df_from_sql["bikes_docked"] = df_from_sql["number_of_docks"] - df_from_sql["available_docks"]

In [23]:
sum(df_from_sql['bikes_docked'])

5740

Finally, let's clean up and close our database connection.

In [24]:
drop_db_query = "DROP DATABASE {db} ".format(db=db_name)

# Drop a database
cursor = con.cursor()
cursor.execute(drop_db_query)
cursor.close()


In [25]:
con.close()



#### As you can see, it is relatively straightforward to move data to and from a relational database in python.